# **Implementation and Explanation**

## Correlation Matrix and Kernel

In [ ]:
import numpy as np
from sklearn.decomposition import KernelPCA, PCA
from typing import Tuple, List

def compute_correlation_matrix(data: np.ndarray, rowvar: bool = False) -> np.ndarray:
    """
    Computes the correlation matrix using PCA.

    Args:
    data (np.ndarray): Input data where columns represent variables and rows represent samples.
    rowvar (bool): Whether the data is row-variate (columns are variables) or column-variate (rows are variables). Default is False.

    Returns:
    np.ndarray: The correlation matrix.
    """
    correlation_matrix = np.corrcoef(data, rowvar=rowvar)
    return correlation_matrix


def compute_kernel_matrix(data: np.ndarray, kernel: str = 'rbf', gamma: float = 0.1) -> np.ndarray:
    """
    Computes the kernel matrix using KernelPCA.

    Args:
    data (np.ndarray): Input data where columns represent variables and rows represent samples.
    kernel (str): Type of kernel to use. Default is 'rbf'.
    gamma (float): Kernel coefficient for rbf, poly, and sigmoid kernels. Default is 0.1.

    Returns:
    np.ndarray: The kernel matrix.
    """
    kpca = KernelPCA(n_components=data.shape[1], kernel=kernel, gamma=gamma)
    kernel_matrix = kpca.fit_transform(data.T)
    return kernel_matrix

def compute_eigenvalues_and_eigenvectors(matrix: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Computes the eigenvalues and eigenvectors of a given matrix.

    Args:
    matrix (np.ndarray): Input matrix.

    Returns:
    Tuple[np.ndarray, np.ndarray]: Eigenvalues and eigenvectors.
    """
    eigenvalues, eigenvectors = np.linalg.eigh(matrix)
    return eigenvalues, eigenvectors

def eigenvalue_analysis(eigenvalues: np.ndarray, threshold: float = 0.95) -> List[int]:
    """
    Perform eigenvalue analysis to identify the set of conflicting objectives along significant principal components.

    Args:
    eigenvalues (np.ndarray): Eigenvalues of the matrix.
    threshold (float): Variance threshold to determine significant components. Default is 0.95.

    Returns:
    List[int]: List of indices of significant principal components.
    """
    total_variance = np.sum(eigenvalues)
    explained_variance_ratio = eigenvalues / total_variance
    cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

    significant_components = np.where(cumulative_variance_ratio >= threshold)[0] + 1
    return significant_components



## Selection of Objectives

In [ ]:
import numpy as np
from typing import List, Tuple

# Continue with the functions we defined earlier

def reduced_correlation_matrix_analysis(corr_matrix: np.ndarray, eigenvalues: np.ndarray, eigenvectors: np.ndarray, T_cor: float = 0.5) -> List[List[int]]:
    """
    Perform Reduced Correlation Matrix (RCM) Analysis to identify identically correlated subsets.

    Args:
    corr_matrix (np.ndarray): Correlation matrix.
    eigenvalues (np.ndarray): Eigenvalues of the matrix.
    eigenvectors (np.ndarray): Eigenvectors of the matrix.
    T_cor (float): Correlation threshold. Default is 0.5.

    Returns:
    List[List[int]]: List of lists where each sublist represents an identically correlated subset.
    """
    num_objs = corr_matrix.shape[0]
    identically_correlated_subsets = []

    for i in range(num_objs):
        subset = [i]
        for j in range(num_objs):
            if i != j and np.abs(corr_matrix[i, j]) >= T_cor:
                subset.append(j)
        if subset not in identically_correlated_subsets:
            identically_correlated_subsets.append(subset)

    return identically_correlated_subsets

def selection_scheme(identically_correlated_subsets: List[List[int]], eigenvectors: np.ndarray) -> List[int]:
    """
    Apply the selection scheme to identify the most significant objective in each identically correlated subset.

    Args:
    identically_correlated_subsets (List[List[int]]): List of identically correlated subsets.
    eigenvectors (np.ndarray): Eigenvectors of the matrix.

    Returns:
    List[int]: List of indices of the most significant objectives.
    """
    selected_objectives = []

    for subset in identically_correlated_subsets:
        subset_eigenvectors = eigenvectors[:, subset]
        subset_variance = np.sum(np.var(subset_eigenvectors, axis=1))
        most_significant_obj_index = subset[np.argmax(subset_variance)]
        selected_objectives.append(most_significant_obj_index)

    return selected_objectives

def compute_error(data: np.ndarray, selected_objectives: List[int]) -> List[float]:
    """
    Compute the error associated with each objective.

    Args:
    data (np.ndarray): Input data where columns represent variables and rows represent samples.
    selected_objectives (List[int]): List of indices of the selected objectives.

    Returns:
    List[float]: List of errors associated with each objective.
    """
    errors = []

    for obj_index in selected_objectives:
        remaining_indices = [i for i in range(data.shape[1]) if i != obj_index]
        reduced_data = data[:, remaining_indices]
        reduced_correlation_matrix = compute_correlation_matrix(reduced_data)
        error = np.trace(reduced_correlation_matrix)
        errors.append(error)

    return errors

# Implement the remaining steps (8 to 12) following your descriptions.

def main(data):
    #data = np.random.rand(5, 50)  # Replace with your actual data

    # Step 6: Reduced Correlation Matrix Analysis
    corr_matrix = compute_correlation_matrix(data)
    eigenvalues, eigenvectors = compute_eigenvalues_and_eigenvectors(corr_matrix)
    T_correlation = 0.5  # Adjust this threshold as needed
    identically_correlated_subsets = reduced_correlation_matrix_analysis(corr_matrix, eigenvalues, eigenvectors, T_correlation)
    print("\nIdentically Correlated Subsets:")
    print(identically_correlated_subsets)

    # Step 7: Selection Scheme
    selected_objectives = selection_scheme(identically_correlated_subsets, eigenvectors)
    print("\nSelected Objectives:")
    print(selected_objectives)

    # Step 8: Compute Error
    errors = compute_error(data, selected_objectives)
    print("\nErrors Associated with Selected Objectives:")
    print(errors)

    # Continue with the implementation of the remaining steps (9 to 12) as described in your document.


# Problems

In [ ]:
pip install git+https://github.com/msu-coinlab/pymop

  Cloning https://github.com/msu-coinlab/pymop to /tmp/pip-req-build-byayq8zz
  Running command git clone --filter=blob:none --quiet https://github.com/msu-coinlab/pymop /tmp/pip-req-build-byayq8zz
  Resolved https://github.com/msu-coinlab/pymop to commit 7b7e789e640126c6d254e86ede5d7f4baad7eaa5
  Preparing metadata (setup.py) ... done
  Created wheel for pymop: filename=pymop-0.2.4-py3-none-any.whl size=43638 sha256=20ac4d4db47fd6f551852833a87c7c770397891be6e31d35e2783a421b0387dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-usm41t_1/wheels/bd/df/1c/0dd14bf65c12caaa4c11bb5ad03d03b802ab48ad29e8eeef72
Successfully built pymop


In [ ]:
import pymop



## DLTZ2

In [ ]:
from math import factorial

PROBLEM = "dtlz2"
NOBJ = 3
K = 10
NDIM = NOBJ + K - 1
P = 12
H = factorial(NOBJ + P - 1) / (factorial(P) * factorial(NOBJ - 1))
BOUND_LOW, BOUND_UP = 0.0, 1.0
#problem = pymop.factory.get_problem(PROBLEM, n_var=NDIM, n_obj=NOBJ)
problem = pymopfactory.get_problem(PROBLEM, n_var=NDIM, n_obj=NOBJ)

In [ ]:
ref_dirs = pymop.factory.get_uniform_weights(100, 3)
dltz2=problem.pareto_front(ref_dirs)

In [ ]:
main(dltz2)


Identically Correlated Subsets:
[[0], [1], [2]]

Selected Objectives:
[0, 1, 2]

Errors Associated with Selected Objectives:
[2.0, 2.0, 2.0]


## DLTZ5

In [ ]:
# Problem definition
PROBLEM = "dtlz5"
NOBJ = 10
K = 1 #nvars - nobj + 1
NDIM = NOBJ + K - 1

# numb. of reference points - directions on the objective space - a parameter for NSGA-III
#P = 5 - 4
#P = 7 - 4
#P = 14 - 4

#H = factorial(NOBJ + P - 1) / (factorial(P) * factorial(NOBJ - 1))
BOUND_LOW, BOUND_UP = 0.0, 1.0
problem = pymop.factory.get_problem(PROBLEM, n_var=NDIM, n_obj=NOBJ)

In [ ]:
#ref_dirs = pymop.factory.get_uniform_weights(100, 3)
dltz5=problem.pareto_front()

Exception: ignored